# Flip-Flop Qubit Coupling Mediated By Microwave Cavity

In this research work, we seek to compute the effective coupling between flip-flop qubits via a long microwave cavity. This is motivated by the fact that we want to build large scale quantum computing architectures but are currently restricted by the range at which we can couple spin qubits. Currently, spin coupling is done via the exchange coupling, which requires wavefunction overlap. What we want is to couple spins at distances much larger than that, in the order of hundred of nanometers, which is possible via mocrowave cavity.

In order to do so, we couple the spins to the donor charge qubit. The charge qubits form electric dipoles which can couple to photons in the cavity.

In [31]:
# Import necessary Python libraries
from sympy import *
from sympy.physics.quantum import Ket, Bra, Dagger, HBar, Operator, Commutator, TensorProduct
from sympy.physics.paulialgebra import Pauli, evaluate_pauli_product
from sympy.matrices import Matrix, diag,eye, Identity
init_printing(scale = 3,use_latex='mathjax',latex_mode='equation')

## 1. Hamiltonian: One Donor, One Photon
We will first start with a single flip-flop qubit and a single cavity mode. In addition, we only consider the vacuum and singly occupied photon state. Later on, we will introduce the second flip-flop qubit and then consider more photons in the cavity.

For now, our Hamiltonian is :
$$
\newcommand{ket}[1]{\left|#1\right\rangle}
\newcommand{bra}[1]{\left\langle#1\right|}
H = \hbar \omega_c a^\dagger a + \sum_j E_j \ket{j}\bra{j} + \frac{1}{2}g_c(a+a^\dagger)(1+Z)
$$
where $Z$ is defined as : 
$$
Z = \sum_{jk} z_{jk}\sigma_j\tau_j 
$$
We use a dressed flip-flop qubit so the $\sigma$ *primarily* affect the charge qubit and the $\tau$ primarily affect the spin states. The $a$ ($a^\dagger$) is the annihilation (creation) operator for the photon, $g_c$ is the coupling strength between the flip-flop qubit and the cavity, and $\omega_c$ is the cavity energy.

In [32]:
# Initialize all of our symbols
hbar=symbols('\hbar')
wc=symbols('\omega_c')
a=MatrixSymbol('a',2,2)
gc=symbols('g_c')
E0,E1,E2,E3=symbols('E_0 E_1 E_2 E_3')
s0=Matrix([[1,0],[0,1]])
s1=Matrix([[0,1],[1,0]])
s2=Matrix([[0,-I],[I,0]])
s3=Matrix([[1,0],[0,-1]])
z01,z03,z10,z11,z13,z22, z30,z31,z33 = symbols('z_01 z_03 z_10 z_11 z_13 z_22 z_30 z_31 z_33')
a=Matrix([[0,1],[0,0]])

In [33]:
# Build our matrices
H_ff = diag(E0,E1,E2,E3)
Z=z03*TensorProduct(eye(2),s3) + z10*TensorProduct(s1,eye(2)) + z30*TensorProduct(s3,eye(2)) + z31*TensorProduct(s3,s1) + z33*TensorProduct(s3,s3)
Htotal1q1p = hbar*wc*TensorProduct(Dagger(a)*a,eye(4)) + TensorProduct(eye(2),H_ff) + gc*TensorProduct(a+Dagger(a),eye(4)+Z)/2
Htotal1q1p

⎡                                                                             
⎢           E₀                          0                           0         
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢            0                          E₁                          0         
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢            0                          0                          E₂         
⎢                                                                             
⎢                                                                             
⎢                                                   

From the above matrix, we can see that the $\ket{0}$ and $\ket{1}$ states are coupled with a strength of $g_cz_{31}/2$ through the creation/annihilation of a photon. This is the spin-photon coupling strength. Unfortunately, The charge-photon coupling strength coupling the $\ket{0}$ and $\ket{2}$ states is much larger at $g_cz_{10}/2$. We also have the terms that don't change the system state. These would introduce dephasing in the system.

## Hamiltonian: Two Donors

Now we extend the system to cover two flip-flop systems. We can now couple these together through the microwave cavity photon.

Our Hilbert space will be arranged in the following order: $H_c \otimes H_0 \otimes H_1$ where the qubits are numbered indexing from 0.

$$
H = \hbar \omega_c a^\dagger a + \sum_{i=L,R}\sum_{j=0}^3 E_j^{(i)} \ket{j}\bra{j}_{(i)} + \sum_{i=L,R}\frac{1}{2}g_c^{(i)}(a+a^\dagger)(1+Z^{(i)})
$$
Under this Hamiltonian, the total state space is $N_{c}\times 16$ which is very large when we consider multiple photon states.

In [36]:
E00,E01,E02,E03,E10,E11,E12,E13=symbols('E_0^(0) E_1^(0) E_2^(0) E_3^(0) E_0^(1) E_1^(1) E_2^(1) E_3^(1)')
z001,z003,z010,z011,z013,z022,z030,z031,z033 = symbols('z_01^(0) z_03^(0) z_10^(0) z_11^(0) z_13^(0) z_22^(0) z_30^(0) z_31^(0) z_33^(0)')
z101,z103,z110,z111,z113,z122,z130,z131,z133 = symbols('z_01^(1) z_03^(1) z_10^(1) z_11^(1) z_13^(1) z_22^(1) z_30^(1) z_31^(1) z_33^(1)')
H_0 = TensorProduct(eye(2),diag(E00,E01,E02,E03),eye(4))
H_1 = TensorProduct(eye(2),eye(4),diag(E10,E11,E12,E13))
Z_0 = TensorProduct(eye(2),z003*TensorProduct(eye(2),s3) + z010*TensorProduct(s1,eye(2)) + z030*TensorProduct(s3,eye(2)) + z031*TensorProduct(s3,s1) + z033*TensorProduct(s3,s3),eye(4))
Z_1 = TensorProduct(eye(2),eye(4),z103*TensorProduct(eye(2),s3) + z110*TensorProduct(s1,eye(2)) + z130*TensorProduct(s3,eye(2)) + z131*TensorProduct(s3,s1) + z133*TensorProduct(s3,s3))
Htotal2q1p = H_0 + H_1 + hbar*wc*TensorProduct(Dagger(a)*a,eye(4),eye(4)) + gc*(eye(32)+Z_0)*TensorProduct(a+Dagger(a),eye(4),eye(4))/2 + gc*(eye(32)+Z_1)*TensorProduct(a+Dagger(a),eye(4),eye(4))/2
Htotal2q1p

                         
₃₁                                                                 g_c⋅z⁽¹⁾₁₀ 
──                                                                 ────────── 
                                                                       2      
                                                                              
_c⋅(-z⁽¹⁾₀₃ + z⁽¹⁾₃₀ - z⁽¹⁾₃₃ + 1)                                            
──────────────────────────────────                                     0      
                2                                                             
                                                                              
                                    g_c⋅(-z⁽⁰⁾₀₃ - z⁽⁰⁾₃₀ + z⁽⁰⁾₃₃ + 1)   g_c⋅
                                    ─────────────────────────────────── + ────
                                                     2                        
                                                                              
₁₀                        

From here, it's not to difficulty to see that a swap-like process between $\ket{01}$ and $\ket{10}$ goes as follows:

1. $\ket{01} \rightarrow \ket{11}$ by absorbing a photon
2. $\ket{11} \rightarrow \ket{10}$ by creating a photon

Or alternatively:

1. $\ket{01} \rightarrow \ket{00}$ by creating a photon
2. $\ket{00} \rightarrow \ket{10}$ by absorbing a photon

There are also two non-energy-conserving processes that would be ignored by applying the rotating wave approximation leading to a madified Jaynes-Cummings model. Which is what we're going to do now.

If we consider that the cavity energy $\omega_c$ is close to the flip-flop energy $\omega_{ff} = (E_1 - E_0)/\hbar$, we can simplify the Hamiltonian greatly. Just so don't lose any important information, we also consider that the dressed charge qubit energy $\omega_{e} = (E_2-E_0)/\hbar$ is close to the flip-flop and cavity energies so that all three can be near resonance. The modified effective Hamiltonian is then:
$$
    H = \hbar \omega_c a^\dagger a + \sum_{i=L,R}\sum_{j=0}^3 E_j^{(i)} \ket{j}\bra{j}_{(i)} + \left\{ a\otimes \left[\sum_{i=L,R} g_{ff}^{(i)}\left(\ket{1}\bra{0}_{(i)}+\ket{3}\bra{2}_{(i)}\right)+g_e^{(i)}\left(\ket{2}\bra{0}_{(i)}+\ket{3}\bra{1}_{(i)}\right)\right] + h.c. \right\}
$$
where $g_{ff} = \frac{1}{2}g_cz_{31}$ and $g_{e} = \frac{1}{2}g_cz_{10}$. This Hamiltonian is separable into blocks of $16\times16$ for values of $n_0$. The states for each of these blocks are:
$$
\begin{align}
    & \ket{n_0-4,33} \\
    & \ket{n_0-3,13} & \ket{n_0-3,23} & \ket{n_0-3,31} & \ket{n_0-3,32} \\
    & \ket{n_0-2,03} & \ket{n_0-2,11} & \ket{n_0-2,12} & \ket{n_0-2,21} & \ket{n_0-2,22} & \ket{n_0-2,30} \\
    & \ket{n_0-1,01} & \ket{n_0-1,02} & \ket{n_0-1,10} & \ket{n_0-1,20} \\
    & \ket{n_0-0,00}
\end{align}
$$
Since the photon occupancy is restricted to be $n>=0$, blocks with $n_0 = 0..3$ are even smaller than $16\times16$. Within each of these blocks, if $|\omega_{ff}-\omega_{c}| \ll |\omega_{e} - \omega{c}|$ we can do another rotating wave approximation to get a reduced Jaynes-Cummings model.

$$
    H = \hbar \omega_c a^\dagger a + \sum_{i=L,R}\sum_{j=0}^1 E_j^{(i)} \ket{j}\bra{j}_{(i)} + \left\{ a\otimes \sum_{i=L,R} g_{ff}^{(i)}\ket{1}\bra{0}_{(i)} + h.c. \right\}
$$
This would further reduce each block down to states:
$$
\begin{align}
    & \ket{n_0-2,11} \\
    & \ket{n_0-1,01} & \ket{n_0-1,10} \\
    & \ket{n_0-0,00}
\end{align}
$$